In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!wget 'https://www.dropbox.com/s/pwg7req7eos0pwq/Subscribers.csv?dl=0'

--2023-08-03 16:40:45--  https://www.dropbox.com/s/pwg7req7eos0pwq/Subscribers.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/pwg7req7eos0pwq/Subscribers.csv [following]
--2023-08-03 16:40:46--  https://www.dropbox.com/s/raw/pwg7req7eos0pwq/Subscribers.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7724b573e59eac37eabe112d89.dl.dropboxusercontent.com/cd/0/inline/CBGWHQ0e53e31HQowCTbdn54Y7Ev71Dr4BYzkufkbWcWEeBRVkWDL82ZpaILwTXfG6PegOAZ4qF8G7Ay1Nt7ebXl_Bif2Y8eQ64KAVDY2TFUX6qZNo0SyXKpOhw6GvdU0jNvGcUw5ulNJ2Pmk2NT-oNQ/file# [following]
--2023-08-03 16:40:47--  https://uc7724b573e59eac37eabe112d89.dl.dropboxusercontent.com/cd/0/inline/CBGWHQ0e53e31HQowCTbdn54Y7Ev71Dr4BYzkufkbWcWEeBRVkWDL82ZpaILwTXfG6PegOAZ4qF8G7Ay1Nt7eb

In [ ]:
data=pd.read_csv('Subscribers.csv?dl=0')

In [ ]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V42,V43,V44,V45,V46,V47,V48,V49,V50,Subscribers
0,-0.211875,743952.92,743952.92,200000.0,743952.92,200000.000,200000.0,-0.069396,943952.92,0.00,...,1.034232e+06,1966.57,1990.82,1752.89,100000.0,147.0,1826.046667,28.0,2557.22,0
1,0.241488,11295310.87,8034290.99,32000.0,8034290.99,18000.000,14000.0,0.167411,7508345.76,1160925.13,...,5.665658e+07,49957.65,11104.54,17711.99,38000.0,19.0,26467.683330,19.0,12807.07,0
2,0.459032,20363.68,317922.94,1650000.0,317922.94,1650000.000,1650000.0,46.068404,1655168.31,0.00,...,2.654720e+04,453.96,9.64,58.83,45072.0,0.0,315.506667,0.0,32.12,1
3,0.000000,856120.91,856120.91,0.0,856120.91,45554.885,0.0,0.000000,856120.91,0.00,...,8.506973e+05,2045.09,2045.09,2011.19,45072.0,30.0,2045.090000,19.0,2029.79,0
4,8.270783,194485.64,412330.33,835000.0,412330.33,835000.000,835000.0,-0.272225,2241.64,78625.00,...,5.572559e+04,70.01,100.60,89.59,31080.0,0.0,294.580000,0.0,26.95,0


In [ ]:
data.shape

(300000, 51)

In [ ]:
y=pd.get_dummies(data['Subscribers'])

In [ ]:
y.head()

,0,1
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


In [ ]:
x=data.drop(['Subscribers'],axis=1)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,
                                               random_state=2)


In [ ]:
x_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)

# drop is set to True to prevent using old index as a column

In [ ]:
st=StandardScaler()

In [ ]:
st.fit(x_train)

StandardScaler()

#Simple Logistic Regression

In [ ]:
weights=tf.Variable(tf.random.normal(shape=(50,2),dtype=tf.float64))
biases=tf.Variable(tf.random.normal(shape=(2,),dtype=tf.float64))

In [ ]:
def logistic_regression(x):
  lr=tf.add(tf.matmul(x,weights),biases)
  return lr

def cross_entropy(y_true,y_pred):
  loss=tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred)
  return tf.reduce_mean(loss)

def accuracy(y_true,y_pred):
  y_true=tf.cast(tf.argmax(y_true,axis=1),dtype=tf.int32)
  preds=tf.cast(tf.argmax(y_pred,axis=1),dtype=tf.int32)
  preds=tf.equal(y_true,preds)
  return tf.reduce_mean(tf.cast(preds,dtype=tf.float32))

def roc_auc(y_true,x):
  y_true=tf.cast(tf.argmax(y_true,axis=1),dtype=tf.int32).numpy()
  y_pred=tf.nn.softmax(logistic_regression(x))[:,1]
  return (roc_auc_score(y_true,y_pred))

def grad(x,y):
  with tf.GradientTape() as tape:
    y_pred=logistic_regression(x)
    loss_val=cross_entropy(y,y_pred)
  return tape.gradient(loss_val,[weights,biases])


In [ ]:
steps=2000

In [ ]:
y_true=tf.cast(tf.argmax(y_test.values,axis=1),dtype=tf.int32)

In [ ]:
preds=tf.cast(tf.argmax(logistic_regression(x_test.values),axis=1),dtype=tf.int32)

In [ ]:
tf.reduce_mean(tf.cast(tf.equal(y_true,preds),dtype=tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=0.8263>

In [ ]:
learning_rate=0.01

for step in range(steps):
  rand_ind=np.random.choice(range(x_train.shape[0]),100)
  outputs=y_train.iloc[rand_ind,:].values
  inputs=x_train.iloc[rand_ind,:].values

  dw,db=grad(inputs,tf.cast(outputs,'float32'))

  weights.assign_sub(learning_rate*dw)
  biases.assign_sub(learning_rate*db)

  if step%50==0:
    print(accuracy(y_test.values,logistic_regression(x_test.values)).numpy(),roc_auc(y_test.values,x_test.values))

0.17165 0.5000100599573458
0.82475 0.5420841229849539
0.82888335 0.5033528182991018
0.8257 0.6298060542601236
0.8354667 0.5385444540195631
0.8279167 0.5167038646077369
0.82608336 0.6508622327115833
0.6928667 0.6865868566557635
0.82783335 0.5067993159213375
0.49551666 0.5246240149066647
0.80366665 0.6481096748155226
0.7474667 0.7029527250621121
0.82891667 0.5027185544977998
0.8211167 0.7507950746323794
0.85765 0.6855825079485777
0.8306 0.5393407829717358
0.84173334 0.6163878373681623
0.85683334 0.6254251724039112
0.855 0.6547666664694662
0.8358333 0.5350704297437939
0.8379833 0.7056610808849565
0.7535833 0.631198136269912
0.594 0.6481209207107416
0.84041667 0.6413835177799517
0.8042667 0.6655627962153366
0.84206665 0.7127449496063932
0.83206666 0.5300253718806152
0.82905 0.6697438680544983
0.8282833 0.4999497002132711
0.82795 0.5035593238830249
0.8276167 0.5163303195766911
0.82413334 0.5486406733207407
0.77713335 0.5897517469802681
0.56255 0.5278237846995164
0.7449167 0.6111085987810716

# Classifier with a hidden layer

In [ ]:
weights_h=tf.Variable(tf.random.truncated_normal(shape=(50,20),dtype=tf.float64))
biases_h=tf.Variable(tf.random.truncated_normal(shape=(20,),dtype=tf.float64))

weights_o=tf.Variable(tf.random.truncated_normal(shape=(20,2),dtype=tf.float64))
biases_o=tf.Variable(tf.random.truncated_normal(shape=(2,),dtype=tf.float64))

In [ ]:
def logistic_regression(x):
  h_o=tf.add(tf.matmul(x,weights_h),biases_h)
  h_o=tf.nn.relu(h_o)
  lr=tf.add(tf.matmul(h_o,weights_o),biases_o)
  return lr

def cross_entropy(y_true,y_pred):
  loss=tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred)
  return tf.reduce_mean(loss)

def accuracy(y_true,y_pred):
  y_true=tf.cast(tf.argmax(y_true,axis=1),dtype=tf.int32)
  preds=tf.cast(tf.argmax(y_pred,axis=1),dtype=tf.int32)
  preds=tf.equal(y_true,preds)
  return tf.reduce_mean(tf.cast(preds,dtype=tf.float32))

def roc_auc(y_true,x):
  y_true=tf.cast(tf.argmax(y_true,axis=1),dtype=tf.int32).numpy()
  y_pred=tf.nn.softmax(logistic_regression(x))[:,1]
  return (roc_auc_score(y_true,y_pred))

def grad(x,y):
  with tf.GradientTape() as tape:
    y_pred=logistic_regression(x)
    loss_val=cross_entropy(y,y_pred)
  return tape.gradient(loss_val,[weights_h,biases_h,weights_o,biases_o])


In [ ]:
steps=2000

In [ ]:
learning_rate=0.01

for step in range(steps):
  rand_ind=np.random.choice(range(x_train.shape[0]),100)
  outputs=y_train.iloc[rand_ind,:].values
  inputs=x_train.iloc[rand_ind,:].values

  dw_h,db_h,dw_o,db_o=grad(inputs,tf.cast(outputs,'float32'))

  weights_h.assign_sub(learning_rate*dw_h)
  biases_h.assign_sub(learning_rate*db_h)
  weights_o.assign_sub(learning_rate*dw_o)
  biases_o.assign_sub(learning_rate*db_o)

  if step%50==0:
    print(accuracy(y_test.values,logistic_regression(x_test.values)).numpy(),roc_auc(y_test.values,x_test.values))

0.17163333 0.5
0.17163333 0.500008313287727
0.17163333 0.5000284332024185
0.17163333 0.5000284332024185
0.8283333 0.5000183732450728
0.8283333 0.5000183732450728
0.8283333 0.5000183732450728
0.82835 0.5000284332024185
0.82835 0.5000384921828798
0.82835 0.5000384921828798
0.82835 0.5000384921828798
0.82835 0.5000384921828798
0.82836664 0.4999632535098545
0.8283167 0.500008313287727
0.8283167 0.500008313287727
0.8283167 0.500008313287727
0.82836664 0.4999715667975815
0.82836664 0.4999715667975815
0.82836664 0.4999715667975815
0.82836664 0.4999715667975815
0.82836664 0.4999715667975815
0.82836664 0.4999715667975815
0.82836664 0.4999715667975815
0.82836664 0.4999715667975815
0.82836664 0.4999715667975815
0.82836664 0.4999715667975815
0.82836664 0.4999715667975815
0.82836664 0.4999715667975815
0.82836664 0.4999715667975815
0.82836664 0.4999715667975815
0.82836664 0.4999715667975815
0.82836664 0.4999715667975815
0.82836664 0.4999715667975815
0.82836664 0.4999715667975815
0.82836664 0.4999715